In [75]:
import numpy as np
import pandas as pd
import random

### Death: Florida and its reference states

In [76]:
all_death = pd.read_csv('../20_intermediate_files/only_od_all_years.csv')
all_death.head()

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,CTYNAME,STNAME
0,"Cochise County, AZ",4003.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,13.0,Cochise County,AZ
1,"Gila County, AZ",4007.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,Gila County,AZ
2,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,411.0,Maricopa County,AZ
3,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Suicide (X60-X64),D2,118.0,Maricopa County,AZ
4,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Undetermined (Y10-Y14),D4,45.0,Maricopa County,AZ


In [77]:
fl_pop = pd.read_csv('../20_intermediate_files/fl_death_pop.csv')
fl_pop.STNAME.unique()

array(['Colorado', 'Florida', 'Louisiana', 'Nevada'], dtype=object)

In [78]:
# select only florida and its reference states
fl_ref_states = ['LA', 'NV', 'CO', 'FL']
fl_death = all_death[all_death['STNAME'].isin(fl_ref_states)]

# rename the abbrevation state name to its full name
fl_death_ = fl_death.replace({'NV': 'Nevada', 'FL': 'Florida', 'LA': 'Louisiana', 'CO': 'Colorado'})

# make sure the state names are the same
assert (fl_death_['STNAME'].unique() == fl_pop['STNAME'].unique()).all()

# rename year
fl_death_ = fl_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
fl_death_ = fl_death_.loc[:, [ 'YEAR', 'STNAME', 'CTYNAME','Deaths']]

In [79]:
# Collapsing the dataset to get total number of deaths per county per year
fl_agg_death = fl_death_.groupby(['YEAR', 'STNAME', 'CTYNAME'], as_index=False)['Deaths'].sum()
fl_agg_death.head()

,YEAR,STNAME,CTYNAME,Deaths
0,2003.0,Colorado,Adams County,42.0
1,2003.0,Colorado,Arapahoe County,45.0
2,2003.0,Colorado,Boulder County,20.0
3,2003.0,Colorado,Denver County,107.0
4,2003.0,Colorado,El Paso County,53.0


In [80]:
# merge the two dataframes
fl_death_pop = pd.merge(fl_pop, fl_agg_death, how='outer', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
fl_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Colorado,Adams County,2000,350961,NaN,left_only
1,Colorado,Alamosa County,2000,14964,NaN,left_only
2,Colorado,Arapahoe County,2000,491701,NaN,left_only
3,Colorado,Archuleta County,2000,10038,NaN,left_only
4,Colorado,Baca County,2000,4501,NaN,left_only


In [81]:
# remove the year where we don't have the opoid death data
min_year = fl_death_['YEAR'].min()
max_year = fl_death_['YEAR'].max()

fl_death_pop = fl_death_pop[(fl_death_pop['YEAR'] >= min_year) & (fl_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
fl_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Colorado,Adams County,2003,378785,42.0,both
1,Colorado,Alamosa County,2003,15125,NaN,left_only
2,Colorado,Arapahoe County,2003,514406,45.0,both
3,Colorado,Archuleta County,2003,11167,NaN,left_only
4,Colorado,Baca County,2003,4106,NaN,left_only
...,...,...,...,...,...,...
2751,Nevada,Pershing County,2015,6620,NaN,left_only
2752,Nevada,Storey County,2015,3881,NaN,left_only
2753,Nevada,Washoe County,2015,442617,84.0,both
2754,Nevada,White Pine County,2015,9875,NaN,left_only


In [82]:
# missing values
fl_death_pop.isna().sum()

STNAME           0
CTYNAME          0
YEAR             0
POPULATION       0
Deaths        1933
_merge           0
dtype: int64

We only have missing values for deaths

In [83]:
# use random number 1-10 to fill the missing values
fl_death_pop['Deaths'] = fl_death_pop['Deaths'].apply(lambda x: np.random.randint(0, 10) if np.isnan(x) else x)

In [84]:
# check whether there is any missing value
assert fl_death_pop.isna().sum().sum() == 0

In [85]:
fl_death_pop = fl_death_pop.drop(columns=['_merge'])
fl_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths
0,Colorado,Adams County,2003,378785,42.0
1,Colorado,Alamosa County,2003,15125,8.0
2,Colorado,Arapahoe County,2003,514406,45.0
3,Colorado,Archuleta County,2003,11167,5.0
4,Colorado,Baca County,2003,4106,8.0


In [86]:
# Calculate the death rate
fl_death_pop['Death Rate (%)'] = 100 * fl_death_pop['Deaths'] / fl_death_pop['POPULATION']
fl_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,Death Rate (%)
0,Colorado,Adams County,2003,378785,42.0,0.011088
1,Colorado,Alamosa County,2003,15125,8.0,0.052893
2,Colorado,Arapahoe County,2003,514406,45.0,0.008748
3,Colorado,Archuleta County,2003,11167,5.0,0.044775
4,Colorado,Baca County,2003,4106,8.0,0.194837


In [87]:
# Add indicator for treatment and control group
fl_death_pop['Indicator'] = fl_death_pop['STNAME'].apply(lambda x: "Treatment" if x == 'Florida' else "Control")
fl_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,Death Rate (%),Indicator
0,Colorado,Adams County,2003,378785,42.0,0.011088,Control
1,Colorado,Alamosa County,2003,15125,8.0,0.052893,Control
2,Colorado,Arapahoe County,2003,514406,45.0,0.008748,Control
3,Colorado,Archuleta County,2003,11167,5.0,0.044775,Control
4,Colorado,Baca County,2003,4106,8.0,0.194837,Control


In [88]:
fl_death_pop.Indicator.value_counts()

Control      1885
Treatment     871
Name: Indicator, dtype: int64

### Death: Texas and its reference states

In [89]:
tx_pop = pd.read_csv('../20_intermediate_files/tx_death_pop.csv')
tx_pop.STNAME.unique()

array(['New York', 'Oregon', 'Texas', 'Wisconsin'], dtype=object)

In [90]:
# select only florida and its reference states
tx_ref_states = ['NY', 'OR', 'TX', 'WI']
tx_death = all_death[all_death['STNAME'].isin(tx_ref_states)]

# rename the abbrevation state name to its full name
tx_death_ = tx_death.replace({'NY': 'New York', 'OR': 'Oregon', 'TX': 'Texas', 'WI': 'Wisconsin'})

# make sure the state names are the same
assert (tx_death_['STNAME'].unique() == tx_pop['STNAME'].unique()).all()

# rename year
tx_death_ = tx_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
tx_death_ = tx_death_.loc[:, [ 'YEAR', 'STNAME', 'CTYNAME','Deaths']]

In [91]:
# Collapsing the dataset to get total number of deaths per county per year
tx_agg_death = tx_death_.groupby(['YEAR', 'STNAME', 'CTYNAME'], as_index=False)['Deaths'].sum()
tx_agg_death.head()

,YEAR,STNAME,CTYNAME,Deaths
0,2003.0,New York,Bronx County,131.0
1,2003.0,New York,Kings County,216.0
2,2003.0,New York,Nassau County,21.0
3,2003.0,New York,New York County,197.0
4,2003.0,New York,Onondaga County,10.0


In [92]:
# merge the two dataframes
tx_death_pop = pd.merge(tx_pop, tx_agg_death, how='outer', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
tx_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,New York,Albany County,2000,294960,NaN,left_only
1,New York,Allegany County,2000,49916,NaN,left_only
2,New York,Bronx County,2000,1333854,NaN,left_only
3,New York,Broome County,2000,200319,NaN,left_only
4,New York,Cattaraugus County,2000,84017,NaN,left_only


In [93]:
# remove the year where we don't have the opoid death data
min_year = tx_death_['YEAR'].min()
max_year = tx_death_['YEAR'].max()

tx_death_pop = tx_death_pop[(tx_death_pop['YEAR'] >= min_year) & (tx_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
tx_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,New York,Albany County,2003,298491,NaN,left_only
1,New York,Allegany County,2003,50220,NaN,left_only
2,New York,Bronx County,2003,1353712,131.0,both
3,New York,Broome County,2003,198364,NaN,left_only
4,New York,Cattaraugus County,2003,82968,NaN,left_only
...,...,...,...,...,...,...
5507,Wisconsin,Waukesha County,2015,396377,34.0,both
5508,Wisconsin,Waupaca County,2015,51684,NaN,left_only
5509,Wisconsin,Waushara County,2015,23966,NaN,left_only
5510,Wisconsin,Winnebago County,2015,169383,22.0,both


In [94]:
# missing values
tx_death_pop.isna().sum()

STNAME           0
CTYNAME          0
YEAR             0
POPULATION       0
Deaths        4587
_merge           0
dtype: int64

We only have missing values for deaths

In [95]:
# use random number 1-10 to fill the missing values
tx_death_pop['Deaths'] = tx_death_pop['Deaths'].apply(lambda x: np.random.randint(0, 10) if np.isnan(x) else x)

In [96]:
# check whether there is any missing value
assert tx_death_pop.isna().sum().sum() == 0

In [97]:
tx_death_pop = tx_death_pop.drop(columns=['_merge'])
tx_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths
0,New York,Albany County,2003,298491,6.0
1,New York,Allegany County,2003,50220,0.0
2,New York,Bronx County,2003,1353712,131.0
3,New York,Broome County,2003,198364,1.0
4,New York,Cattaraugus County,2003,82968,7.0


In [98]:
# Calculate the death rate
tx_death_pop['Death Rate (%)'] = 100 * tx_death_pop['Deaths'] / tx_death_pop['POPULATION']
tx_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,Death Rate (%)
0,New York,Albany County,2003,298491,6.0,0.002010
1,New York,Allegany County,2003,50220,0.0,0.000000
2,New York,Bronx County,2003,1353712,131.0,0.009677
3,New York,Broome County,2003,198364,1.0,0.000504
4,New York,Cattaraugus County,2003,82968,7.0,0.008437


In [99]:
# Add indicator for treatment and control group
tx_death_pop['Indicator'] = tx_death_pop['STNAME'].apply(lambda x: "Treatment" if x == 'Texas' else "Control")
tx_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,Death Rate (%),Indicator
0,New York,Albany County,2003,298491,6.0,0.002010,Control
1,New York,Allegany County,2003,50220,0.0,0.000000,Control
2,New York,Bronx County,2003,1353712,131.0,0.009677,Control
3,New York,Broome County,2003,198364,1.0,0.000504,Control
4,New York,Cattaraugus County,2003,82968,7.0,0.008437,Control


In [100]:
tx_death_pop.Indicator.value_counts()

Treatment    3302
Control      2210
Name: Indicator, dtype: int64

### Death: Washington and its reference states

In [101]:
wa_pop = pd.read_csv('../20_intermediate_files/wa_death_pop.csv')
wa_pop.STNAME.unique()

array(['Hawaii', 'Oklahoma', 'Oregon', 'Washington'], dtype=object)

In [102]:
# select only florida and its reference states
wa_ref_states = ['HI', 'OK', 'OR', 'WA']
wa_death = all_death[all_death['STNAME'].isin(wa_ref_states)]

# rename the abbrevation state name to its full name
wa_death_ = wa_death.replace({'HI': 'Hawaii', 'OK': 'Oklahoma', 'OR': 'Oregon', 'WA': 'Washington'})

# make sure the state names are the same
assert (wa_death_['STNAME'].unique() == wa_pop['STNAME'].unique()).all()

# rename year
wa_death_ = wa_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
wa_death_ = wa_death_.loc[:, [ 'YEAR', 'STNAME', 'CTYNAME','Deaths']]

In [103]:
# Collapsing the dataset to get total number of deaths per county per year
wa_agg_death = wa_death_.groupby(['YEAR', 'STNAME', 'CTYNAME'], as_index=False)['Deaths'].sum()
wa_agg_death.head()

,YEAR,STNAME,CTYNAME,Deaths
0,2003.0,Hawaii,Hawaii County,11.0
1,2003.0,Hawaii,Honolulu County,62.0
2,2003.0,Oklahoma,Cleveland County,15.0
3,2003.0,Oklahoma,Oklahoma County,75.0
4,2003.0,Oklahoma,Tulsa County,100.0


In [104]:
# merge the two dataframes
wa_death_pop = pd.merge(wa_pop, wa_agg_death, how='outer', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
wa_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Hawaii,Hawaii County,2000,149095,NaN,left_only
1,Hawaii,Honolulu County,2000,875061,NaN,left_only
2,Hawaii,Kalawao County,2000,145,NaN,left_only
3,Hawaii,Kauai County,2000,58511,NaN,left_only
4,Hawaii,Maui County,2000,128754,NaN,left_only


In [105]:
# remove the year where we don't have the opoid death data
min_year = wa_death_['YEAR'].min()
max_year = wa_death_['YEAR'].max()

wa_death_pop = wa_death_pop[(wa_death_pop['YEAR'] >= min_year) & (wa_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
wa_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Hawaii,Hawaii County,2003,156340,11.0,both
1,Hawaii,Honolulu County,2003,888026,62.0,both
2,Hawaii,Kalawao County,2003,129,NaN,left_only
3,Hawaii,Kauai County,2003,60061,NaN,left_only
4,Hawaii,Maui County,2003,134742,NaN,left_only
...,...,...,...,...,...,...
2036,Washington,Wahkiakum County,2015,3989,NaN,left_only
2037,Washington,Walla Walla County,2015,59970,NaN,left_only
2038,Washington,Whatcom County,2015,211942,23.0,both
2039,Washington,Whitman County,2015,48224,NaN,left_only


In [106]:
# missing values
wa_death_pop.isna().sum()

STNAME           0
CTYNAME          0
YEAR             0
POPULATION       0
Deaths        1571
_merge           0
dtype: int64

We only have missing values for deaths

In [107]:
# use random number 1-10 to fill the missing values
wa_death_pop['Deaths'] = wa_death_pop['Deaths'].apply(lambda x: np.random.randint(0, 10) if np.isnan(x) else x)

In [108]:
# check whether there is any missing value
assert wa_death_pop.isna().sum().sum() == 0

In [109]:
wa_death_pop = wa_death_pop.drop(columns=['_merge'])
wa_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths
0,Hawaii,Hawaii County,2003,156340,11.0
1,Hawaii,Honolulu County,2003,888026,62.0
2,Hawaii,Kalawao County,2003,129,1.0
3,Hawaii,Kauai County,2003,60061,5.0
4,Hawaii,Maui County,2003,134742,6.0


In [110]:
# Calculate the death rate
wa_death_pop['Death Rate (%)'] = 100 * wa_death_pop['Deaths'] / wa_death_pop['POPULATION']
wa_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,Death Rate (%)
0,Hawaii,Hawaii County,2003,156340,11.0,0.007036
1,Hawaii,Honolulu County,2003,888026,62.0,0.006982
2,Hawaii,Kalawao County,2003,129,1.0,0.775194
3,Hawaii,Kauai County,2003,60061,5.0,0.008325
4,Hawaii,Maui County,2003,134742,6.0,0.004453


In [111]:
# Add indicator for treatment and control group
wa_death_pop['Indicator'] = wa_death_pop['STNAME'].apply(lambda x: "Treatment" if x == 'Washington' else "Control")
wa_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,Death Rate (%),Indicator
0,Hawaii,Hawaii County,2003,156340,11.0,0.007036,Control
1,Hawaii,Honolulu County,2003,888026,62.0,0.006982,Control
2,Hawaii,Kalawao County,2003,129,1.0,0.775194,Control
3,Hawaii,Kauai County,2003,60061,5.0,0.008325,Control
4,Hawaii,Maui County,2003,134742,6.0,0.004453,Control


In [112]:
wa_death_pop.Indicator.value_counts()

Control      1534
Treatment     507
Name: Indicator, dtype: int64